# CipherRecognizer
#### Jan Jasiński & Jan Wilczek

###### Technologia Mowy AGH 2017

### Wstęp

Celem projektu "CipherRecognizer" jest stworzenie prostego systemu rozpoznawania nagranych nazw cyfr zaimplementowany w języku Python.
Link do repozytorium: https://github.com/Jasinsk/CipherRecognizer

### Raport 1 (16.11.17-23.11.17)

1. Założenia

   Do naszego projektu wybraliśmy parametryzację MFCC sygnału mowy - do jej wyznaczania używamy biblioteki python_speech_features.

   Jako klasyfikatora chcemy użyć odpowiednio wyszkolonej sieci neuronowej. Zdecydowaliśmy się na klasyfikator MLPClassifier z biblioteki sklearn.neural_network. Na jego właściwym ustawieniu opierać się będzie znaczna część procesu dopasowania modelu nauki klasyfikatora.
   
   Wejściem sieci neuronowej będzie superwektor o długości 13 + 13x13 = 182, złożony z uśrednionego po czasie wektora parametrów MFCC wraz z kolejnymi rzędami macierzy kowariancji ustawionymi jeden za drugim w ciąg.
   
2. Architektura systemu

   Cały nasz projekt opiera się na klasach. Obecnie ich lista wraz z metodami wygląda następująco:
   
   A. `GlobalParameters` (niezaimplementowana)
      - klasa w modelu singleton zawierająca wszystkie parametry, którymi można regulować (dostrajać) system.
      
   B. `WaveFile` - klasa odpowiedzialna za odczytanie danych z pliku wave.
    * `__init__(self, filepath)`  - konstruktor, filepath określa ścieżkę dostępu do danego pliku z nagraniem cyfry.
    * `data(self, normalize=True)` - zwraca dane z pliku audio jako listę z wartościami typu float w zakresie <-1,1>.
    
   C. `MFCCParametrizer` - klasa odpowiedzialna za parametryzację MFCC sygnału.
    * `__init__(self,winlen=0.025,winstep=0.01,numcep=13,nfilt=26,nfft=512,preemph=0.97,ceplifter=22,               appendEnergy=True)` - konstruktor ustawiający parametry wyznaczania MFCC zgodnie z kolejnymi argumentami.
    * `parameters(self, signal, samplerate)` - zwraca macierz wyznaczonych parametrów z podanego sygnału.
	* `super_vector(self, signal, samplerate)` - zwraca superwektor parametrów wyznaczonych na podstawie danego sygnału.
    
   D. `ANNClassifier` - klasa będąca opakowaniem dla sieci neuronowej.
    * `__init__(self, nb_hidden_layers, nb_neurons_in_layer, activation_function='relu', solver='lbfgs', nb_iterations=200)` - konstruktor klasyfikatora z zadanymi parametrami. W przyszłości zmienimy nieco tę implementację, żeby w każdej warstwie sieci nie było tyle samo neuronów.
    * `train(self, training_input_data, training_output_data)` - wytrenowuje model podając kolejne wektory z wejściami i odpowiadające im wyjścia.
    * `predict(self, test_input_data)` - oblicza logarytmiczne prawdopodobieństwa każdej klasy (w naszym przypadku jest 10 klas odpowiadających 10 cyfrom) i zwraca ich wektor.
    
   E. `ResultHolder` - klasa odpowiadająca za zbieranie wyników (prawdopodobieństw) w testach kros-walidacji i obliczania skuteczności dla zadanych parametrów. Rezultaty można następnie eksportować do pliku .xls.
    * `__init__(self, classes)` - konstruktor przekazujący nazwy klas. Przekazywanym wektorem klas powinien być `ANNClassifier.MLPClassifier.classes_`.
    * `add_result(self, prediction_vector, correct_result)` - dodaje kolejne wyniki otrzymywane w wyniku wywołania metody `predict()` klasy `ANNClassifier` i zapisuje wraz z przekazanym poprawnym wynikiem.
    * `error_rate(self)`- oblicza procent błędów wszystkich dodanych wyników.
    * `write_results_to_excel_file(self, filename, sheet)` - wypisuje wszystkie wyniki wraz z ich poprawnymi odpowiedziami, poprawnością (za odpowiedź uznaje się klasę z największym prawdopodobieństwem) oraz końcowy procent błędów do pliku .xls.
    * `__is_correct(self, index)` - prywatna metoda zwracająca wartość logiczną mówiącą o tym, czy odpowiedź wyniku o podanym indeksie jest poprawna.
  
   F. `Recognizer` (niezaimplementowana) - główna klasa odpowiedzialna za pętle testów, zmianę parametrów itp. Jej metody nie są jeszcze sprecyzowane.

   G. `ConfigurationManager` (niezaimplementowana) - klasa służąca do generowania konfiguracji w sposób deterministyczny (dla przykładu: jeśli podamy liczbę plików testowych równą 10 każde wywołanie metody `__generate_configurations` powinno wygenerować te same konfiguracje).
    * `__init__(self, foldername)` - konstruktor z przekazywaną nazwą folderu z plikami audio.
    * `__generate_configurations(self, nb_test_files)` - generuje konfiguracje do kros-walidacji o zadanej liczbie testów w każdym zbiorze.
    * `nb_configurations(self)` - zwraca liczbę konfiguracji.
    * `test_data(self, configuration_id)` - zwraca ścieżki plików wybrane do testu w konfiguracji o danym indeksie.
    * `training_data(self, configuration_id)` - zwraca ścieżki plików wybrane do treningu w konfiguracji o danym indeksie.
    * `output_configurations_to_file(self, filename)` - wypisuje konfiguracje do pliku.
   
3. Postęp prac
 
    Udało nam się do tej pory zaimplementować wstępnie klasy ANNClassifier, MFCCParametrizer, ResultHandler i WavFile wraz z testami. Oznacza to, że w praktyce jesteśmy w stanie wczytać plik, otworzyć go, wyliczyć parametry MFCC i następnie wprowadzić je jako dane wejściowe sieci neuronowej. Klasy te są gotowe do zastosowania we wstępnej implementacji pętli treningowo-testowej służącej do wybrania odpowiednich parametrów systemu. Kiedy uda nam się połączyc ze sobą wsystkie te funkcje to będziemy musieli jeszcze zaimplementować jakąś formę automatyzacji trenowania i testowania systemu, gdyż obecnie musielibyśmy wszystkie ścieżki plików wprowadzać ręcznie. Warto byłoby także stworzyć mechanizm zapisywania do pliku wartości parametrów otrzymanych po dokonaniu MFCC, żeby móc wczytywać te pliki przy kolejnych wykonaniach programu, aby uniknąć każdorazowego wyliczania ich z plików dźwiękowych. 
    Największą trudnością będzie dobór wielkości i liczby warstw w sieci neuronowej, ponieważ brak nam doświadczenia w tej dziedzinie. Spodziewamy się zatem, iż będzie to etap wymagający znaczącej ilości prób i błędów. Zależnie od potrzeby możliwe jest, iż do wstępnej optymalizacji wykorzystamy funkcję, która będzie zmieniać te paramtery sieci, trenować ją i następnie testować, co pozwoli nam zobaczyć jak dla zmian tych parametrów będzie zmieniać się prawidłowość otrzymywanej odpowiedzi przy testowaniu systemu. 
    Dotychczas nie spotkaliśmy sie z nadmiernymi trudnościami jeżeli chodzi o pisane programy. Wszelkie problemy udawało się rozwiązać wspólnymi siłami. Nie mielimy jednak jeszcze możliwości sprawdzenia klasy ANNClasifier, która w razie ewentualnych błędów przy wykonywaniu mogłaby sprawić najwięcej problemu w związku z faktem, iż żaden z nas nigdy nie korzystał z sieci neuronowych i w pisaniu musimy opierać się wyłącznie na dokumentacji oraz poradach z Internetu.
    Obecnie planujemy aby do końca tego tygodnia pracy mieć zakończone i poprawione wszystkie poszczególne funkcje w klasach wraz z sprawdzeniem ich skuteczności po połączeniu w wspólny program. To pozwoli nam w weekend skupić się na tworzeniu automatyzacji trenowania i testowania oraz na następnej interpretacji otrzymanych danych i wyborze optymalnych parametrów całego programu.

   




***
### Raport 2 (23.11.17 - 30.11.17)

1. Zmiany w architekturze

   W wyniku przemyśleń i występowania pewnych problemów postanowiliśmy zrezygnować z początkowego projektu klasy `ConfigurationManager`, w której użytkownik określa liczbę nagrań testowych (a w istocie osób, których nagrania miały być w całości testowe) na rzecz stałej konfiguracji leave-one-out, która wyklucza nagrania jednej osoby z procesu treningowego w każdej konfiguracji. Po tych poprawkach udało się zaimplementować działającą klasę `ConfigurationManager`, która z podanego folderu z danymi treningowymi tworzy słownik osób z przypisanymi im nagraniami i w metodach `training_data()` i `test_data()` zwraca odpowiednio wszystkie pliki z wyłączeniem jednej osoby (jej alfabetyczny indeks odpowiada numerowi konfiguracji `configuration_id`) i pliki nagrane przez osobę wybraną do testów kroswalidacji.
    Zrezygnowaliśmy z klasy `GlobalParameters` na rzecz mniej zautomatyzowanego podejścia oraz z klasy `Recognizer` - ta ostatnia została zastąpiona różnymi odmianami pliku `main.py.`. W związku z problemami w używanych bibliotekach, usunięta została normalizacja danych w klasie WavFile.
    
2. Implementacja pętli treningowo-testowej

   Główna pętla treningowo testowa, do której wprowadzaliśmy parametry i w wyniku której otrzymywaliśmy stopę błędu ErrorRate (% błędów z wszystkich konfiguracji) znajduje się w pliku `main.py`. W nim tworzone są wszystkie instancje klas naszego systemu z odpowiednimi parametrami.
    Najpierw tworzony jest obiekt klasy `ConfigurationManager`, do którego przekazywana jest nazwa folderu z plikami testowymi. Następnie tworzona jest instancja klasy `MFCCParametrizer` z odpowiednimi parametrami.
    Następnie w pętli zawierającej różne parametry sieci neuronowej tworzony był obiekt klasy `ANNClassifier`, który następnie był trenowany na danych dostarczonych przez instancję `ConfigurationManager`, odczytanych przez osobne obiekty `WavFile` i sparametryzowanych przez instancję `MFCCParametrizer`.
    Pod koniec pętli to samo było wykonywane dla danych testowych z tym, że wywoływana była metoda `predict` klasyfikatora, a wyniki zapisywane były przez osobne obiekty klasy `ResultHandler` (jedna dla jednego zestawu parametrów sieci i klasyfikatora).
    
3. Zmiany parametrów sieci neuronowej

   Pierwszymi parametrami, które wyznaczyliśmy, były funkcja aktywacji i optymalizator wag. Najlepszą kombinacją tych dwóch jest logistyczna funkcja aktywacji i optymalizator Adam. Następnie skupiliśmy się na liczbie warstw i neuronów w każdej warstwie. Z następującego wpisu: https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw zdecydowaliśmy się najpierw na jedną warstwę, a kiedy ją zoptymalizowaliśmy, dodaliśmy kolejną. Ostatecznie pierwsza warstwa ma 240, a druga 120 neuronów. Dodanie trzeciej warstwy pogarsza skuteczność systemu.
   Próbowaliśmy dopasować parametr &alpha; i liczbę iteracji, ale żadne z nich nie dały znaczącej poprawy, więc przypisaliśmy im wartość domyślną.   
   
4. Zmiany ustawień parametryzacji MFCC

    Manipulowaliśmy wartościami takimi jak długość okna, liczba współczynników, zakładkowanie i liftering. Jedynym parametrem, który ostatecznie zmieniliśmy z korzyścią dla wyników, było `appendEnergy=False` - można się spodziewać, że energia wszystkich nagrań będzie podobna i nie niesie ona istotnej informacji dla systemu, dlatego można z niej zrezygnować.
      
5. Serializacja
    
    Ostatnim krokiem było napisanie skryptu serializującego wyszkolony na wszystkich danych obiekt klasyfikatora i zapisanie go w pliku oraz skryptu deserializującego ten obiekt i używającego go w procesie rozpoznawania cyfr w podanych nagraniach. Na końcu nasz skrypt zapisuje wyniki do pliku .csv w zadanym przez prowadzącego formacie.
    
6. Podsumowanie wyników
    
    W testach kroswalidacji nasz system osiągnął stopę błędu poniżej 15%. Nie jest to zadowalający wynik, ale nie daliśmy rady dalej usprawnić systemu. Wydaje się, że jedyną możliwą do przeprowadzenia zmianą z korzyścią dla wyników byłoby użycie do treningu pojedynczych ramek zamiast superwektora. Największy problem wystąpił dla cyfr 9 i 5 - klasyfikator prawdopodobnie nie potrafił ustalić odpowiednich wag dla nich, gdyż zazwyczaj błędna odpowiedź dla jednej z nich polegała na wytypowaniu drugiej. Wynika to prawdopodobnie stąd, że "dziewięć" jest najdłuższym słowem w tym zbiorze (pod względem czasu wypowiadania), a uśredniony wektor MFCC wraz z macierzą kowariancji nie zawiera dokładnej informacji o przebiegu czasowym. Po usunięciu tej informacji brzmienia cyfr 5 i 9 stają sie bardzo zblizone do siebie, pod względem brzmienia nastepujących po sobie fonemów. 
    
7. Werdykt
   
   System jest gotowy do zewnętrznych testów.
   